# Creating A Facial Detection and Recognition Model
# Idris Nemsia
In this program, I build and train a Siamese Neural Network for attendance recording. Below are the references I used while building this program.
## References
To build my model, I have consulted some references to guide my coding process.
[1] : J. Loy. Implementing a Facial Recognition System with Neural Networks. Neural Network Projects with Python. Packt Publishing, February 2019.
[2] : Build a Facial Recognition App // Deep Learning Project // Paper2Code Series. https://www.youtube.com/playlist?list=PLgNJO2hghbmhHuhURAGbe6KWpiYZt0AMH

## Initial Preparations Section
These section includes the installation of some of our needed modules. It also imports the modules we will be using.

In [1]:
# Installing Tensorflow and opencv
# !pip install --user tensorflow==2.10.1 opencv-python
#!pip install keras

In [4]:
# Imports
import cv2
import os
import numpy as np
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Input

## Loading our data
We will be using ORL AT&T Dataset of Faces, taken at the Olivetti Research Laboratory in Cambridge, UK. It includes 40 people, each with 10 pictures with varying facial expressions.

In this section, we initialize X_train, Y_train, X_test, Y_test. The X lists represent the pictures of the individuals, the Y labels are their identifiers.
We import the folder containing the dataset of images. The folder contains 40 folders, each containing pictures of one person. We split the folders into 35 to 5 folders for training and testing data. We convert these images into numpy arrays containing numerical values after preprocessing them. We add the numpy arrays of images to our X,Y lists accordingly.

In [5]:
# Initialize empty X,Y training and test arrays
X_train, Y_train = [], []
X_test, Y_test = [], []
# Define the dataset path
dataset_path = "dataset"
# List of DirEntry Objects. These objects contain path attributes.
dataset = os.listdir(dataset_path)
dataset = sorted(dataset)
# Our training data is 1-35, test data is 35-40
# Loop through the folders inside the dataset
for folder_index in range(1,41):
    person_name = dataset[folder_index]
    person_folder_path = dataset_path + "/" + person_name
    person_folder = os.listdir(person_folder_path)
    # Loop through images in each person's folder
    for img_name in person_folder:
        img_path = person_folder_path + '/' + img_name
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        img = np.array(img)
        img = np.expand_dims(img, axis=2)
        # Splitting dataset_path
        if folder_index <= 35:
            X, Y = X_train, Y_train
        else:
            X, Y = X_test, Y_test

        # Normalize the pixel values to [0, 1]
        img = img / 255.0
        X.append(img)
        Y.append(person_name)
        # Display the preprocessed image (Used for testing)
        #cv2.imshow('Preprocessed Image', img)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()

# Check the shape of the image
img_shape = X_train[0].shape
print(img_shape)
# Get image width and height values
img_height, img_width = img_shape[0], img_shape[1]



(112, 92, 1)


## Creating X_train, Y_train, X_test, Y_test using pairs of images and their outputs (0 or 1)

In [6]:
from helper_functions import create_X_pairs
def show_images(list_of_pairs):
    count = 0
    for pair in list_of_pairs:
        count += 1
        image1, image2 = pair[0], pair[1]
        combined_image = np.zeros((img_height,img_width*2, 3))
        combined_image[:, :img_width] = image1
        combined_image[:, img_width:] = image2
        print(combined_image)
        cv2.imshow('Combined Image', combined_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        if(count == 2):
            break

# Creating our training and data sets for pairs of images
X_pairs_train, Y_pairs_train = create_X_pairs(X_train, 35, 150)
X_pairs_test, Y_pairs_test = create_X_pairs(X_test, 5, 30)
# Turning our training and testing sets into numpy arrays
X_pairs_train = np.array(X_pairs_train)
Y_pairs_train = np.array(Y_pairs_train)
X_pairs_test = np.array(X_pairs_test)
Y_pairs_test = np.array(Y_pairs_test)
show_images(X_pairs_train)


[[[0.19607843 0.19607843 0.19607843]
  [0.19215686 0.19215686 0.19215686]
  [0.19607843 0.19607843 0.19607843]
  ...
  [0.14901961 0.14901961 0.14901961]
  [0.17254902 0.17254902 0.17254902]
  [0.15686275 0.15686275 0.15686275]]

 [[0.21568627 0.21568627 0.21568627]
  [0.18039216 0.18039216 0.18039216]
  [0.19607843 0.19607843 0.19607843]
  ...
  [0.15686275 0.15686275 0.15686275]
  [0.16470588 0.16470588 0.16470588]
  [0.15686275 0.15686275 0.15686275]]

 [[0.19607843 0.19607843 0.19607843]
  [0.19215686 0.19215686 0.19215686]
  [0.19215686 0.19215686 0.19215686]
  ...
  [0.16470588 0.16470588 0.16470588]
  [0.16862745 0.16862745 0.16862745]
  [0.15686275 0.15686275 0.15686275]]

 ...

 [[0.71764706 0.71764706 0.71764706]
  [0.64705882 0.64705882 0.64705882]
  [0.56862745 0.56862745 0.56862745]
  ...
  [0.5372549  0.5372549  0.5372549 ]
  [0.50196078 0.50196078 0.50196078]
  [0.6        0.6        0.6       ]]

 [[0.52156863 0.52156863 0.52156863]
  [0.58039216 0.58039216 0.58039216]


## Creating our Model
### Creating the feature generating model
The layers of this section follow the layers explained in [2]. However, more modifications will be made to this code as Max Pooling layers are incomplete. This is causing the loss value to be NaN during the training of my model.

In [7]:
# I am referencing the code used to build the model used in [1]
feature_generation_network = Sequential()
feature_generation_network.add(Conv2D(filters=128,
 kernel_size=(3,3), activation='relu',
                 input_shape=img_shape))
feature_generation_network.add(MaxPooling2D())
feature_generation_network.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
feature_generation_network.add(Flatten())
feature_generation_network.add(Dense(units=128, activation='sigmoid'))

In [8]:
# Get the summary of the structure of the model
feature_generation_network.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 110, 90, 128)      1280      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 55, 45, 128)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 53, 43, 64)        73792     
                                                                 
 flatten (Flatten)           (None, 145856)            0         
                                                                 
 dense (Dense)               (None, 128)               18669696  
                                                                 
Total params: 18,744,768
Trainable params: 18,744,768
Non-trainable params: 0
____________________________________________

### Defining the inputs and outputs
The outputs here refer to the network used to generate features inside our siamese neural network for each input. The two outputs of the feature_generation_network from input1 and input2 will then be compared by calculating the Euclidean distance between them.

In [9]:
input1, input2 = Input(shape=img_shape), Input(shape=img_shape)
# Share the single output layer for both inputs
output1, output2 = feature_generation_network(input1), feature_generation_network(input2)

## Defining the output of the siamese_network: Distance between our two inital outputs

In [10]:
# References [1]
from helper_functions import euclidean_distance
from keras.layers import Lambda
# Creating the final layer of the SNN
distance = Lambda(euclidean_distance, output_shape=(1,))([output1, output2])



### Initialize Siamese Neural Network Model with inputs and outputs

In [51]:
siamese_network = Model((input1, input2), distance)

### Get Summary of our model

In [52]:
siamese_network.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 112, 92, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 input_2 (InputLayer)           [(None, 112, 92, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 sequential (Sequential)        (None, 128)          18744768    ['input_1[0][0]',                
                                                                  'input_2[0][0]']          

## Training our model

### Compile our model

In [53]:
from helper_functions import contrastive_loss
siamese_network.compile(loss=contrastive_loss, optimizer='adam')

# Exporting our model to be integrated into a web application

In [16]:
# Creating numpy arrays for the first and second element of each pair
X1_train = np.array([pair[0] for pair in X_pairs_train])
X2_train = np.array([pair[1] for pair in X_pairs_train])
# Training the model
siamese_network.fit(x=[X1_train, X2_train], y = Y_pairs_train, batch_size = 3, epochs=10)

Epoch 1/10
50/50 [==============================] - 8s 150ms/step - loss: 0.1751
Epoch 2/10
50/50 [==============================] - 8s 158ms/step - loss: 0.2403
Epoch 3/10
50/50 [==============================] - 8s 154ms/step - loss: 0.4086
Epoch 4/10
50/50 [==============================] - 8s 158ms/step - loss: 0.1616
Epoch 5/10
50/50 [==============================] - 8s 155ms/step - loss: 0.1532
Epoch 6/10
50/50 [==============================] - 8s 159ms/step - loss: 0.1065
Epoch 7/10
50/50 [==============================] - 8s 154ms/step - loss: 0.0778
Epoch 8/10
50/50 [==============================] - 8s 160ms/step - loss: 0.0581
Epoch 9/10
50/50 [==============================] - 9s 170ms/step - loss: 0.0426
Epoch 10/10
50/50 [==============================] - 8s 167ms/step - loss: 0.0298


## Testing our model

In [17]:
X1_test = np.array([pair[0] for pair in X_pairs_test])
X2_test = np.array([pair[1] for pair in X_pairs_test])
Y_predict = siamese_network.predict([X1_test,X2_test])
print(Y_predict)

1/1 [==============================] - 0s 340ms/step
[[0.97025144]
 [0.89391136]
 [0.4291342 ]
 [1.5194192 ]
 [0.32079366]
 [0.5642118 ]
 [0.13655846]
 [1.0836482 ]
 [0.6714111 ]
 [0.6571402 ]
 [0.58907366]
 [0.77412087]
 [0.43421918]
 [0.8392836 ]
 [0.42481506]
 [0.3643305 ]
 [0.24260598]
 [0.6177844 ]
 [0.15254879]
 [0.8523122 ]
 [0.35152063]
 [0.8523122 ]
 [0.532136  ]
 [0.85002744]
 [0.21059269]
 [0.8926866 ]
 [0.31522262]
 [1.0228065 ]
 [0.24538383]
 [0.88241714]]


In [18]:
# Evaluating accuracy
from sklearn.metrics import accuracy_score
for i in range(len(Y_predict)):
    if Y_predict[i] < 0.5:
        Y_predict[i] = 0
    else:
        Y_predict[i] = 1

# Calculate accuracy
accuracy = accuracy_score(Y_pairs_test, Y_predict)
print("Accuracy:", accuracy)

Accuracy: 0.16666666666666666


## Saving our model
This part was causing me errors. The model was not getting saved correctly and I could not load it in another file. This is why I continued my work in this file instead of using a more structured file structure.

In [20]:
#siamese_network.save("siamese_network.h5");

In [55]:
# Creating an interface to test our model

In [56]:
from StudentsList import StudentsList
from Camera import Camera
import os
import tensorflow as tf

students_list = StudentsList(os.getcwd() + "/students")
students_list.load()
camera = Camera(students_list, siamese_network)
op = False
while True:
    if not op:
        print('''Attendance Recorder Program
    --------------------Options-------------------
    'a': Show list of students
    'b': Show list of present students
    'c': Register student
    'd': Delete student
    'e': Take attendance (Not functional)
    'f': Check model accuracy
    'g': Reset
    'p': Quit
    ' ': continue
    ----------------------------------------------
    ''')
    op = True
    option = input('Input your option:')
    if option == 'a':
        for student in students_list.get_list_of_students():
            print(student.get_name())
    elif option == 'b':
        for student in students_list.get_present_students():
            print(student.get_name())
    elif option == 'c':
        name = input("What is the name of the student: ")
        students_list.add_student(name)
        camera.add_student_pictures(name)
        print(f"Student {name} successfully registered")
        print()
    elif option == 'd':
        name = input("What is the name of the student: ")
        students_list.delete_student(name)
        print(f"Student {name} successfully deleted")
        print()
    elif option == 'e':
        print("Unfortunately, this option is currently not functional")
        #camera.take_attendance()
        #print(f"Present students: {students_list.get_present_students()}")
    elif option == 'f':
        for student in students_list:
            student.set_attendance(False)
    elif option == 'p':
        break
    elif option == ' ':
        op = False
    else:
        print("Option is not valid")



Bella's folder already exists
Britz's folder already exists
Giselle's folder already exists
Idris's folder already exists
Jackson's folder already exists
Attendance Recorder Program
    --------------------Options-------------------
    'a': Show list of students
    'b': Show list of present students
    'c': Register student
    'd': Delete student
    'e': Take attendance (Not functional)
    'f': Check model accuracy
    'g': Reset
    'p': Quit
    ' ': continue
    ----------------------------------------------
    
